In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import CSVLogger
import LoadDataset as LD
import numpy as np
import scipy.misc
from scipy.misc import imresize
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

Using TensorFlow backend.


In [9]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 6
print(epochs)

6


In [3]:
n_categories = 3
categories = np.array(["c0", "c1"])

load_data = LD.LoadData('/netscratch/data/state_farm/imgs/train') # give the directory address.
X_dat ,Y_dat = load_data.generate_data()

X_dat, Y_dat = load_data.shuffle_data(X_dat,Y_dat)

Dataset directory added
['c6', 'c4', 'c5', 'c8', 'c1', 'c0', 'c7', 'c3', 'c9', 'c2']


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


c6  directory images are fetched!!! 
c4  directory images are fetched!!! 
c5  directory images are fetched!!! 
c8  directory images are fetched!!! 
c1  directory images are fetched!!! 
c0  directory images are fetched!!! 
c7  directory images are fetched!!! 
c3  directory images are fetched!!! 
c9  directory images are fetched!!! 
c2  directory images are fetched!!! 


In [4]:
X_dat[1].shape

(227, 227, 3)

In [4]:
X = np.array(X_dat)
Y = np.array(Y_dat)
X.shape

(22424, 227, 227, 3)

In [23]:
X[1]

array([[[0.09427444, 0.14525483, 0.11780385],
        [0.09005317, 0.14103357, 0.11358259],
        [0.10114227, 0.15212266, 0.12467168],
        ...,
        [0.84191137, 0.99703965, 0.86481437],
        [0.85719327, 0.98088044, 0.90917564],
        [0.91226964, 0.99761728, 0.98213568]],

       [[0.0937635 , 0.14474389, 0.11729291],
        [0.1061674 , 0.15714779, 0.12969681],
        [0.09922882, 0.15020921, 0.12275823],
        ...,
        [0.46304942, 0.6242138 , 0.47761658],
        [0.13655326, 0.22857291, 0.15548239],
        [0.6518957 , 0.72247225, 0.69901029]],

       [[0.10076877, 0.15174916, 0.12429818],
        [0.10980392, 0.16078431, 0.13333333],
        [0.10850652, 0.15948691, 0.13203593],
        ...,
        [0.37452537, 0.53341975, 0.38300426],
        [0.002043  , 0.09703028, 0.00539447],
        [0.03310631, 0.0946777 , 0.04624927]],

       ...,

       [[0.0238079 , 0.02772946, 0.00812162],
        [0.0204433 , 0.02436486, 0.00475702],
        [0.01876997, 0

In [27]:
def pca(img, alpha_std=0.1):
    orig_img = img.astype(float).copy()
#     img = img / 255.0  # rescale to 0 to 1 range
    # flatten image to columns of RGB
    img_rs = img.reshape(-1, 3)
    # img_rs shape (640000, 3)
    img_centered = img_rs - np.mean(img_rs, axis=0)
    # paper says 3x3 covariance matrix
    img_cov = np.cov(img_centered, rowvar=False)
    # eigen values and eigen vectors
    eig_vals, eig_vecs = np.linalg.eigh(img_cov)
    # sort values and vector
    sort_perm = eig_vals[::-1].argsort()
    eig_vals[::-1].sort()
    eig_vecs = eig_vecs[:, sort_perm]

    # get [p1, p2, p3]
    m1 = np.column_stack((eig_vecs))

    # get 3x1 matrix of eigen values multiplied by random variable draw from normal
    # distribution with mean of 0 and standard deviation of 0.1
    m2 = np.zeros((3, 1))
    # according to the paper alpha should only be draw once per augmentation (not once per channel)
    alpha = np.random.normal(0, alpha_std)

    # broad cast to speed things up
    m2[:, 0] = alpha * eig_vals[:]

    # this is the vector that we're going to add to each pixel in a moment
    add_vect = np.matrix(m1) * np.matrix(m2)

    for idx in range(3):   # RGB
        orig_img[..., idx] += add_vect[idx]
    orig_img = np.clip(orig_img, 0.0, 255.0)
    orig_img = orig_img.astype(np.uint8)
    return orig_img


In [5]:
(x_train, y_train) = (X[0:20000], Y[0:20000])
(x_test, y_test) = (X[20000:], Y[20000:])
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height, img_width, channel)

# x_train = pca(x_train)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(x_train.shape)

227 227 3
(20000, 227, 227, 3)


In [40]:
x_train[5]

array([[[0.04705882, 0.05098039, 0.03137255],
        [0.04705882, 0.05098039, 0.03137255],
        [0.04705882, 0.05098039, 0.03137255],
        ...,
        [0.92358988, 1.        , 0.99904777],
        [0.99714088, 0.99714088, 0.99154358],
        [0.99537013, 1.        , 0.97611644]],

       [[0.04705882, 0.05098039, 0.03137255],
        [0.04705882, 0.05098039, 0.03137255],
        [0.04705882, 0.05098039, 0.03137255],
        ...,
        [0.92358988, 1.        , 1.        ],
        [0.99714088, 0.99642712, 0.99836428],
        [0.99537013, 0.99724924, 0.98483123]],

       [[0.04705882, 0.05098039, 0.03137255],
        [0.04705882, 0.05098039, 0.03137255],
        [0.04705882, 0.05098039, 0.03137255],
        ...,
        [0.92358988, 0.99860758, 1.        ],
        [0.99714088, 0.99321931, 1.        ],
        [0.99537013, 0.99607843, 0.99837816]],

       ...,

       [[0.03425343, 0.0356614 , 0.0505098 ],
        [0.01621215, 0.03057062, 0.04663835],
        [0.00281073, 0

In [10]:
# # for cifar10 dataset
# # Load CIFAR10 Data
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# print(x_train.shape)

# img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# # convert to one hot encoing
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)
# print(y_train.shape)

(50000, 32, 32, 3)
(50000, 10)


In [12]:
# AlexNet Define the Model
model = Sequential()
# model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='valid', input_shape=(img_height, img_width, channel,)))
# for original Alexnet
model.add(Conv2D(96, (3,3), strides=(2,2), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(256, (5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(38, (3,3), activation='relu', padding='same'))
model.add(Conv2D(38, (3,3), activation='relu', padding='same'))
model.add(Conv2D(25, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [13]:

# print the model summary
model.summary()



# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Train the Model

csv_logger = CSVLogger('log.csv', append=True, separator=';')

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1, callbacks=[csv_logger],
                    validation_data=(x_test, y_test))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 114, 114, 96)      2688      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 57, 57, 96)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 57, 57, 96)        384       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 57, 57, 256)       614656    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 28, 28, 256)       0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 28, 28, 256)       1024      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 28, 28, 38)        87590     
__________

In [14]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.30159104711348467
Test accuracy: 0.9306930693069307
